In [1]:
#import selenium libraries
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.common.by import By

#import web scrapy librarires
from urllib.request import urlopen
from bs4 import BeautifulSoup
from lxml import html
import requests

#import data libraries
import pandas as pd
import numpy as np

#import system libraries
import time
import datetime
import sys
import random
import io
import math
from Tools import *
import telegram_send

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\enesy\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\enesy\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\enesy\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\enesy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
#file=File.ReadFromFile(r"files/Stock/News/Links_AMD.txt")


In [3]:
#Functions
#if there is a popup close it
def passPopUp(driver):
    try:
        popup=driver.find_element_by_xpath("//button[@id='_evh-ric-c']")

        print("popup: ",popup)

        #.find_element_by_id("gb_23")
        while popup is not None:
            popup.click()                
            
    except:
        error="\n"+str(datetime.datetime.now())+" tarihinde Popup kapatma esnasında hata alındı: "+ str(sys.exc_info()[0])
        print("Unexpected error:", sys.exc_info()[0])
        File.log(error)
        pass

    #dictionary ye kayıt ekleme
def set_key(dictionary,key,value):
    if key not in dictionary:
        dictionary[key]=value
    elif type(dictionary[key])==list:
        dictionary[key].append(value)
    else:
        dictionary[key]=[dictionary[key],value]
#<a id="close" role="button" tabindex="0" aria-label="Close feedback form" ng-if="showClose || iframeActive" ng-click=" $root.$emit('form:closeSurvey'); " class="ng-scope">×</a>

In [4]:
BASE_DATE=pd.to_datetime('2021-03-07')
#FIVE_YEARS_AGO_DATE= BASE_DATE - datetime.timedelta(days=5*365)
FIVE_YEARS_AGO_DATE= BASE_DATE - datetime.timedelta(days=1000)
FIVE_YEARS_AGO_DATE=datetime.datetime.strftime(FIVE_YEARS_AGO_DATE, "%b %d, %Y")
print("Formatted Date: ",FIVE_YEARS_AGO_DATE)

Formatted Date:  Jun 11, 2018


In [5]:
#headers
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36'}
#chrome driver options
chrome_options=Options()
chrome_options.add_argument('--no-sandbox')
driver = webdriver.Chrome(r'C:\Users\enesy\OneDrive\Okul\Tez\Codes\chromedriver', chrome_options=chrome_options)
driver.implicitly_wait(10)

delay=2

stock_names=["FB","TSLA","NFLX","MSFT","TWTR"]#,"AAPL,"AMD","AMZN","BABA","FB","GOOG","MSFT","NFLX","NVDA","TSLA","TSM","TWTR"]
news_links={}
errors={}

driver.get("https://www.nasdaq.com/market-activity/stocks/"+str(stock_names[0])+"/news-headlines")
time.sleep(10)

for stock in stock_names:
    telegram_send.send(messages=["İşlenen stock: --> "+str(stock)+"\nİşlem zamanı: --> "+str(datetime.datetime.now().strftime("%x %X"))])
    try:
        driver.get("https://www.nasdaq.com/market-activity/stocks/"+str(stock)+"/news-headlines")
        passPopUp(driver)

        recentList = driver.find_elements_by_xpath("//div[@class='symbol-back-to-overview']") 
        for listt in recentList :
                driver.execute_script("arguments[0].scrollIntoView();", listt )
        page_all=[p.text for p in driver.find_elements_by_xpath('//button[@class="pagination__page"]')]
        page=int(page_all[-1])
        #page=5
        s=str(stock)+" için işlenecek sayfa sayısı = "+str(page)
        print(s)


         #consts
        BASE_DATE=pd.to_datetime('2021-03-01')
        FIVE_YEARS_AGO_DATE= BASE_DATE - datetime.timedelta(days=3*365)
        #FIVE_YEARS_AGO_DATE= BASE_DATE - datetime.timedelta(days=5)
        FIVE_YEARS_AGO_DATE=datetime.datetime.strftime(FIVE_YEARS_AGO_DATE, "%b %d, %Y")
        print("Formatted Date: ",FIVE_YEARS_AGO_DATE)
        last_date=FIVE_YEARS_AGO_DATE


        for i in range(1,page+1):
            err=[]
            newsLinks=[]
            page_dates=[dates.text for dates in driver.find_elements_by_xpath('//span[@class="quote-news-headlines__date"]')]

            if str(last_date.upper()) in page_dates:
                k=str(i) + " . sayfada işlem bitti."
                print(k)
                break
            lnk=[]
            lnk=[links.get_attribute("href") for links in driver.find_elements_by_xpath('//a[@class="quote-news-headlines__link"]')]
            newsLinks.append(lnk)
            time.sleep(delay)
            try:
                element=WebDriverWait(driver,delay).until(expected_conditions.presence_of_element_located((By.CLASS_NAME, "pagination__next")))
                Scrap.act(driver)
            except:
                print("Unexpected error:", sys.exc_info()[0])
                error="\n"+str(datetime.datetime.now())+" tarihinde sayfa geçişleri esnasında  "+str(sys.exc_info()[0])+"  hatası alındı."
                err.append(error)
                pass

            set_key(news_links,stock,newsLinks)
            set_key(errors,stock,err)

    
    except:
        print("For döngüsünde alınan hata: ",sys.exc_info()[0])
        pass
driver.close()
finishTime=datetime.datetime.now().strftime("%x %X")
telegram_send.send(messages=["Verileri çekme işlemi tamamlandı",str(finishTime)])

C:\Users\enesy\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:6: DeprecationWarning: use options instead of chrome_options
  


popup:  <selenium.webdriver.remote.webelement.WebElement (session="6a5775ed57d6bcd31ae779de597c3fed", element="a5d1301c-95ca-4960-b771-31fb5cd9b475")>
Unexpected error: <class 'selenium.common.exceptions.StaleElementReferenceException'>
FB için işlenecek sayfa sayısı = 1250
Formatted Date:  Mar 02, 2018
For döngüsünde alınan hata:  <class 'selenium.common.exceptions.StaleElementReferenceException'>
popup:  <selenium.webdriver.remote.webelement.WebElement (session="6a5775ed57d6bcd31ae779de597c3fed", element="d5fce912-bfe5-4710-807c-eb29c39226ec")>
Unexpected error: <class 'selenium.common.exceptions.StaleElementReferenceException'>
TSLA için işlenecek sayfa sayısı = 1250
Formatted Date:  Mar 02, 2018
For döngüsünde alınan hata:  <class 'selenium.common.exceptions.StaleElementReferenceException'>
popup:  <selenium.webdriver.remote.webelement.WebElement (session="6a5775ed57d6bcd31ae779de597c3fed", element="d4199ceb-6790-4576-8959-a8141c9c8d31")>
Unexpected error: <class 'selenium.common.e

TypeError: unhashable type: 'list'

{'word': [[1, 1], [1, 1]], 'word1': [[2, 2], [2, 2]]}



# Sayfalardaki linkleri toplar
total_page_dates=[]
def getLinks(delay,last_date,stock):
    #headers
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36'}
    #chrome driver options
    chrome_options=Options()
    chrome_options.add_argument('--no-sandbox')
    driver = webdriver.Chrome(r'C:\Users\enesy\OneDrive\Okul\Tez\Codes\chromedriver', chrome_options=chrome_options)
    driver.implicitly_wait(5)
    driver.get("https://www.nasdaq.com/market-activity/stocks/"+str(stock)+"/news-headlines")
    passPopUp()
    
    err=[]
    newsLinks=[]
    recentList = driver.find_elements_by_xpath("//div[@class='symbol-back-to-overview']") 
    for listt in recentList :
        driver.execute_script("arguments[0].scrollIntoView();", listt )
    
    page_all=[p.text for p in driver.find_elements_by_xpath('//button[@class="pagination__page"]')]
    
    page_dates=[dates.text for dates in driver.find_elements_by_xpath('//span[@class="quote-news-headlines__date"]')]
    #page=page_all[-1]
    page=5
    total_page_dates.append(page_dates)
    
   
    
    for i in range(1,page+1):
        
        s=str(page) + " adet sayfa işlenecek."
        print(s)
        if i%10==0:
            print("Sayfa sayısı: ",i)
        #page_date=driver.find_elements_by_xpath('//span[@class="quote-news-headlines__date"]').text

        if str(last_date) in page_dates:
            k=str(i) + " . sayfada işlem bitti."
            print(k)
            break
        
        lnk=[]
        lnk=[links.get_attribute("href") for links in driver.find_elements_by_xpath('//a[@class="quote-news-headlines__link"]')]
        newsLinks.append(lnk)
        time.sleep(delay)
        
        try:
            element=WebDriverWait(driver,delay).until(expected_conditions.presence_of_element_located((By.CLASS_NAME, "pagination__next")))
            Scrap.act(driver)
        except:
            print("Unexpected error:", sys.exc_info()[0])
            error="\n"+str(datetime.datetime.now())+" tarihinde sayfa geçişleri esnasında  "+str(sys.exc_info()[0])+"  hatası alındı."
            err.append(error)
            pass
        finally:
            driver.close()
    return newsLinks,err


In [6]:
start="\n"+" ************************************* İşlemler başladı ************************************************"
startDate=datetime.datetime.now()
File.log(start)


In [7]:

"""
stock_names=["AAPL","AMD"] #,"AMZN","BABA","FB","GOOG","MSFT","NFLX","NVDA","TSLA","TSM","TWTR"]
news_links={}
errors={}
for stock in stock_names:
    try:
        print("Processing stock name: ",str(stock))
        time.sleep(5)
        newsLinks,err=getLinks(delay=1,last_date=str(FIVE_YEARS_AGO_DATE),stock=stock)
        news_links[stock]=newsLinks
        errors[stock]=err
    except:
        error="\n"+str(datetime.datetime.now())+" tarihinde sayfa açma esnasında hata alındı: "+ str(sys.exc_info()[0])
        print("Unexpected error:", sys.exc_info()[0])
        File.log(error)
        pass
"""

'\nstock_names=["AAPL","AMD"] #,"AMZN","BABA","FB","GOOG","MSFT","NFLX","NVDA","TSLA","TSM","TWTR"]\nnews_links={}\nerrors={}\nfor stock in stock_names:\n    try:\n        print("Processing stock name: ",str(stock))\n        time.sleep(5)\n        newsLinks,err=getLinks(delay=1,last_date=str(FIVE_YEARS_AGO_DATE),stock=stock)\n        news_links[stock]=newsLinks\n        errors[stock]=err\n    except:\n        error="\n"+str(datetime.datetime.now())+" tarihinde sayfa açma esnasında hata alındı: "+ str(sys.exc_info()[0])\n        print("Unexpected error:", sys.exc_info()[0])\n        File.log(error)\n        pass\n'

In [8]:
#calculate running time
endDate=datetime.datetime.now()
seconds=(endDate-startDate).total_seconds()
minute, second = divmod(seconds, 60)
hour, minute = divmod(minute, 60)


elapsedTime="\nLinklerin indirilme süresi: "+str(hour)+" saat "+str(minute)+" dakika "+str(second)+"saniye sürmüştür"
processDateTime="\n"+str(startDate)+" tarihinde başladı\n "+str(endDate)+" tarihinde bitti."
#write log
str_logs=elapsedTime+"\n"+processDateTime+"\n"+"ERRORS: \n"+str(err)
File.WriteToFile("logLinks.txt",str_logs)
Mail.send_mail("Veri çekme işlemi tamamlandı - "+str(startDate),elapsedTime)
if len(err)>0:
    Mail.send_mail("Errors-Links - "+str(startDate),str(err))
else:
    Mail.send_mail("No Error-Links - "+str(startDate), "Link elde etme işlemleri hatasız tamamlandı")


Mail gönderildi
Hata:  <class 'smtplib.SMTPDataError'>


In [9]:
#news=np.array(newsLinks)
#xNews=np.array(news).flatten()

for key,value in news_links.items():
    s=r"files\Stock\News\Links\Links_"+str(key)+".txt"
    for i in value:
        File.WriteToFile(s,str(i)+"\n")


In [10]:
finish="\n"+" *********************************** İşlemler tamamlandı ***********************************************"
#l.log(finish)

In [11]:
#xNews

In [35]:

f=File.ReadFromFile(r"files\Stock\News\Links_AMZN.txt")

In [12]:
telegram_send.send(messages=["Bilgisayar kapatılıyor......\n"])

In [ ]:
try:
    import os
    os.system(r'rundll32.exe powrprof.dll,SetSuspendState Hibernate')
except:
    print("Hata: ",sys.exc_info()[0])



In [4]:
try:
    import os
    os.system("shutdown /h") #hibernate
except:
    print("Hata: ",sys.exc_info()[0])